## Working With Large Datasets in Pandas

Is it possible to work with big data in Python without PySpark and the like?! Yes, it can be done with Pandas too... But there are techniques to avoid disaster!

In [1]:
import pandas as pd

Understanding the sizes:

In [2]:
!wc ./datasets/* \
| tr -s ' ' \
| head -n -1 \
| awk '{ printf "Filepath: %s\nLines: %i\tSize in MB: %.2f\n", $4, $1, $3/1024^2 }'

Filepath: ./datasets/car_evaluation.csv
Lines: 1728	Size in MB: 0.05
Filepath: ./datasets/titanic.csv
Lines: 892	Size in MB: 0.06
Filepath: ./datasets/used_car_prices.csv
Lines: 4501	Size in MB: 0.28
Filepath: ./datasets/youtube_trends_us.csv
Lines: 40950	Size in MB: 59.85


### Technique No. 1: Only load the required columns

When you want to do sentiment analysis on video titles, it is not necessary to read the video id, and even its description...

In [3]:
df = pd.read_csv('./datasets/youtube_trends_us.csv', usecols=['title', 'tags', 'views', 'likes'])
df.head()

,title,tags,views,likes
0,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,748374,57527
1,The Trump Presidency: Last Week Tonight with J...,"last week tonight trump presidency|""last week ...",2418783,97185
2,"Racist Superman | Rudy Mancuso, King Bach & Le...","racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033
3,Nickelback Lyrics: Real or Fake?,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172
4,I Dare You: GOING BALD!?,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235


In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   40949 non-null  object
 1   tags    40949 non-null  object
 2   views   40949 non-null  int64 
 3   likes   40949 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 18.8 MB


More details of the size of each column (in bytes):

In [5]:
df.memory_usage(deep=True)

Index         128
title     4628488
tags     14452593
views      327592
likes      327592
dtype: int64

### Technique No. 2: Chunking

In [6]:
chunks = pd.read_csv('./datasets/youtube_trends_us.csv', chunksize=5000)
type(chunks)

pandas.io.parsers.readers.TextFileReader

In [7]:
for chunk in chunks:
    chunk['title'] = chunk['title'].apply(str.title)
    chunk['tags'] = chunk['tags'].apply(lambda x: x.split('|'))
    display(chunk)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,We Want To Talk About Our Marriage,CaseyNeistat,22,2017-11-13T17:13:01.000Z,[SHANtell martin],748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight With J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"[last week tonight trump presidency, ""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"[racist superman, ""rudy"", ""mancuso"", ""king"", ""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real Or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"[rhett and link, ""gmm"", ""good mythical morning...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: Going Bald!?,nigahiga,24,2017-11-12T18:01:41.000Z,"[ryan, ""higa"", ""higatv"", ""nigahiga"", ""i dare y...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0qpE8IHz3go,17.08.12,Youtubers React To Their Old Youtube Channel P...,FBE,24,2017-11-30T22:00:00.000Z,"[Reactception, ""Old Channels"", ""React to Thems...",1152361,34441,949,3048,https://i.ytimg.com/vi/0qpE8IHz3go/default.jpg,False,False,False,CLICK TO SUBSCRIBE TO THE YOUTUBERS IN THIS EP...
4996,osc1DbNpNXE,17.08.12,Chris Stapleton - A Simple Song (Audio),ChrisStapletonVEVO,10,2017-12-01T05:00:03.000Z,"[Chris, ""Stapleton"", ""Simple"", ""Song"", ""Mercur...",271722,3642,178,162,https://i.ytimg.com/vi/osc1DbNpNXE/default.jpg,False,False,False,Available Now From A Room: Volume 2 \nDownload...
4997,6ZfuNTqbHE8,17.08.12,Marvel Studios' Avengers: Infinity War Officia...,Marvel Entertainment,24,2017-11-29T13:26:24.000Z,"[marvel, ""comics"", ""comic books"", ""nerdy"", ""ge...",91933007,2625661,53709,350458,https://i.ytimg.com/vi/6ZfuNTqbHE8/default.jpg,False,False,False,There was an idea… Avengers: Infinity War. In ...
4998,qaivW5joQrk,17.08.12,Prince Harry And Meghan Markle Get The Alison ...,This Morning,24,2017-12-01T13:33:16.000Z,"[this morning, ""interview"", ""holly willoughby""...",229871,1678,271,0,https://i.ytimg.com/vi/qaivW5joQrk/default.jpg,True,False,False,Subscribe now for more! http://bit.ly/1JM41yF\...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
5000,9v_rtaye2yY,17.09.12,"Migos, Nicki Minaj, Cardi B - Motorsport",MigosVEVO,10,2017-12-08T02:12:48.000Z,"[Migos, ""Nicki Minaj"", ""Nicki"", ""Minaj"", ""Card...",4514272,238395,9020,35027,https://i.ytimg.com/vi/9v_rtaye2yY/default.jpg,False,False,False,Listen to MotorSport (feat. Nicki Minaj & Card...
5001,vn9mMeWcgoM,17.09.12,Jurassic World: Fallen Kingdom - Official Trai...,Universal Pictures,24,2017-12-08T01:44:25.000Z,"[Jurassic World, ""Fallen Kingdom"", ""Jurassic P...",18184886,399200,17473,58902,https://i.ytimg.com/vi/vn9mMeWcgoM/default.jpg,False,False,False,Jurassic World: Fallen Kingdom\nIn Theaters Ju...
5002,JBDWIGpKnwM,17.09.12,Dude Vs. Wild - Nevada Mountains,nigahiga,24,2017-12-08T19:21:58.000Z,"[ryan, ""higa"", ""higatv"", ""nigahiga"", ""running ...",1236676,96359,1123,11682,https://i.ytimg.com/vi/JBDWIGpKnwM/default.jpg,False,False,False,"Parody of one of my favorite shows, Running Wi..."
5003,aj8mN_7Apcw,17.09.12,Alita: Battle Angel | Official Trailer [Hd] | ...,20th Century Fox,1,2017-12-08T19:30:09.000Z,"[Trailer, ""alita"", ""battle angel"", ""alita: bat...",2841875,18079,2484,7459,https://i.ytimg.com/vi/aj8mN_7Apcw/default.jpg,False,False,False,From visionary filmmakers James Cameron and Ro...
5004,Bs5Ml7_2wT0,17.09.12,The Game Awards - Full Show With Death Strandi...,thegameawards,20,2017-12-08T08:23:49.000Z,"[the game awards, ""game awards 2017"", ""videoga...",1287342,26557,1467,1905,https://i.ytimg.com/vi/Bs5Ml7_2wT0/default.jpg,False,False,False,"Awards, world premieres and new game announcem..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,zLJsJR0O6VI,18.02.01,2016 Vs 2017,Smosh,23,2017-12-20T17:00:07.000Z,"[smosh, ""2016 vs 2017"", ""vs"", ""smosh vs"", ""ant...",1782889,66165,5100,5414,https://i.ytimg.com/vi/zLJsJR0O6VI/default.jpg,False,False,False,"In 2016 so much has changed, but there are als..."
9996,IU-Fo-QEpOI,18.02.01,Dude Perfect Face Off | What'S In The Box,Whistle Sports,17,2017-12-20T23:00:00.000Z,"[dude perfect, ""dude perfect christmas"", ""dude...",3095325,40946,1082,2472,https://i.ytimg.com/vi/IU-Fo-QEpOI/default.jpg,False,False,False,This Twin Battle is sponsored by Dollar Shave ...
9997,6hb0GOOFQwI,18.02.01,Mst3K (2017) Yule Log,MYSTERY SCIENCE THEATER 3000,23,2017-12-22T00:53:08.000Z,"[mst3k, ""mystery science theater"", ""tom servo""...",24296,828,63,82,https://i.ytimg.com/vi/6hb0GOOFQwI/default.jpg,False,False,False,Enjoy this MST3K Yule Log!\n\nWelcome to the o...
9998,YEGhKwpEKEw,18.02.01,Titanic Transformation! The Rose Look Feat. Gu...,Lisa Eldridge,26,2017-12-21T17:03:31.000Z,"[Lisa Eldridge, ""Lisa Elridge"", ""Lisa Eldrige""...",476030,20712,254,1130,https://i.ytimg.com/vi/YEGhKwpEKEw/default.jpg,False,False,False,Follow me on instagram here http://instagram.c...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
10000,QwZT7T-TXT0,18.03.01,So Sorry.,Logan Paul Vlogs,24,2018-01-02T16:42:21.000Z,"[logan paul vlog, ""logan paul"", ""logan"", ""paul...",13305605,835378,629120,733373,https://i.ytimg.com/vi/QwZT7T-TXT0/default.jpg,False,False,False,NaN
10001,bVU-MmJZFFA,18.03.01,Justin Timberlake - Introducing Man Of The Woods,justintimberlakeVEVO,10,2018-01-02T15:00:04.000Z,"[Justin, ""Timberlake"", ""Man"", ""Of"", ""The"", ""Wo...",725305,21236,1053,2861,https://i.ytimg.com/vi/bVU-MmJZFFA/default.jpg,False,False,False,Man Of The Woods \nFebruary 2nd \nNew Song Ja...
10002,8KgiaRPxRnQ,18.03.01,Developer Update | Happy New Year | Overwatch,PlayOverwatch,20,2018-01-02T18:00:01.000Z,"[“Developer Update , Happy New Year , Overwat...",780084,39946,1089,10203,https://i.ytimg.com/vi/8KgiaRPxRnQ/default.jpg,False,False,False,From our family of heroes to yours: Happy New ...
10003,8V08Gu1DOgE,18.03.01,Fifty Shades Freed - Mrs. Grey Will See You No...,Fifty Shades,24,2018-01-02T02:30:47.000Z,"[Fifty Shades of Grey, ""Official"", ""Teaser"", ""...",2130659,28119,4721,2933,https://i.ytimg.com/vi/8V08Gu1DOgE/default.jpg,False,False,False,#FiftyShadesFreed\n--\nFacebook: https://www.f...
10004,vUiE_0GPqKE,18.03.01,"Børns, Lana Del Rey - God Save Our Young Blood...",BØRNSmusicVEVO,10,2018-01-02T17:00:06.000Z,"[BØRNS, ""Lana"", ""Del"", ""Rey"", ""God"", ""Save"", ""...",285261,29057,470,2764,https://i.ytimg.com/vi/vUiE_0GPqKE/default.jpg,False,False,False,“God Save Our Young Blood” featuring Lana Del ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,5g20u8EfMdg,18.29.01,What If You Never Ate Fruits And Vegetables?,Life Noggin,27,2018-01-22T14:00:01.000Z,"[life noggin, ""life noggin youtube"", ""youtube ...",616440,19998,483,5534,https://i.ytimg.com/vi/5g20u8EfMdg/default.jpg,False,False,False,Your parents were right: You NEED to eat your ...
14996,oZm_N6aSU0E,18.29.01,Entire Breaking Bad Series In 1 Minute,Esther Bellido,24,2018-01-20T20:17:41.000Z,"[breaking, ""bad"", ""series"", ""Heisenberg"", ""net...",707170,9867,277,183,https://i.ytimg.com/vi/oZm_N6aSU0E/default.jpg,False,False,False,Breaking Bad cumple 10 años y este es nuestro ...
14997,IzQwbRdh5Ts,18.29.01,High Fidelity Mixtape Rules,marjolijn2,28,2015-03-29T18:11:39.000Z,[Mixtape],31505,191,1,4,https://i.ytimg.com/vi/IzQwbRdh5Ts/default.jpg,False,False,False,How to make the perfect mixtape?
14998,YNpCXG66_PA,18.29.01,We Share Our Home With 14 Bears | Beast Buddies,Barcroft Animals,15,2018-01-23T00:00:03.000Z,"[amazing news, ""amazing stories"", ""amazing sto...",47862,1064,26,108,https://i.ytimg.com/vi/YNpCXG66_PA/default.jpg,False,False,False,SUBSCRIBE to Barcroft Animals: http://bit.ly/2...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
15000,0cZLo3AAan4,18.30.01,"Dj Khaled, Rihanna - Wild Thoughts (2018 Live ...",Super Netvid,10,2018-01-29T03:11:56.000Z,[[none]],3014847,42556,2791,0,https://i.ytimg.com/vi/0cZLo3AAan4/default.jpg,True,False,False,Zac Efron - Jennifer Lopez - Demi Lovato - Bey...
15001,DjbysQgJ64Y,18.30.01,"Hillary Clinton, Cardi B & More Audition For F...",The Late Late Show with James Corden,24,2018-01-29T03:02:44.000Z,"[grammy, ""grammys"", ""grammy awards"", ""grammy a...",1725057,34607,23353,14563,https://i.ytimg.com/vi/DjbysQgJ64Y/default.jpg,False,False,False,"On the hunt for a spoken word GRAMMY Award, Ja..."
15002,ytsZFYMGl0w,18.30.01,Childish Gambino Grammy Awards Performance 2018,Shyrynne,22,2018-01-29T01:44:05.000Z,"[childish gambino, ""jd mccrary"", ""grammy's"", ""...",1035381,25233,598,2412,https://i.ytimg.com/vi/ytsZFYMGl0w/default.jpg,False,False,False,"Childish Gambino performing Terrified, with JD..."
15003,u7KZrt_cHH0,18.30.01,Unsane | Official Trailer | In Theaters March 23,Bleecker Street,1,2018-01-29T17:30:35.000Z,"[bleecker street, ""bleecker street media"", ""bl...",1267983,2504,159,463,https://i.ytimg.com/vi/u7KZrt_cHH0/default.jpg,False,False,False,Official Site: http://www.Unsane.movie\nLIKE u...
15004,TdcS2LMQ5Ck,18.30.01,Bruno Mars Wins Record Of The Year | Acceptanc...,Recording Academy / GRAMMYs,10,2018-01-29T06:40:28.000Z,"[Grammy, ""2018"", ""Grammys"", ""Performances"", ""S...",656196,8312,1749,1721,https://i.ytimg.com/vi/TdcS2LMQ5Ck/default.jpg,False,False,False,SEE THE WINNERS LIST\nhttp://bit.ly/2GsJoLK\n\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,kIHhVkkHV2k,18.24.02,Double Rainbow Unicorn Apple Pie | How To Cook...,How To Cook That,26,2018-02-23T08:30:11.000Z,"[double rainbow, ""double rainbow unicorn apple...",113160,4597,144,708,https://i.ytimg.com/vi/kIHhVkkHV2k/default.jpg,False,False,False,Watch more Rainbow Desserts: https://www.youtu...
19996,2wbc8x-RHiY,18.24.02,The First Time We Met!,ShibSibs,22,2018-02-23T02:31:14.000Z,"[shibsibs, ""maia shibutani"", ""alex shibutani"",...",107663,12088,29,1053,https://i.ytimg.com/vi/2wbc8x-RHiY/default.jpg,False,False,False,This is some home video shot by our Dad of the...
19997,RWzZeA2GMsk,18.24.02,Bishop Briggs - White Flag (Audio),BishopBriggsVEVO,10,2018-02-23T05:00:01.000Z,"[Bishop, ""Briggs"", ""White"", ""Flag"", ""Teleport""...",41956,4980,37,417,https://i.ytimg.com/vi/RWzZeA2GMsk/default.jpg,False,False,False,Music video by Bishop Briggs performing White ...
19998,2Yz_kqxegrs,18.24.02,Troye Sivan & Ariana Grande Working Together!,BBC Radio 1,10,2018-02-23T08:30:00.000Z,"[Troye Sivan, ""Ariana Grande"", ""My My My"", ""BB...",41761,3149,38,251,https://i.ytimg.com/vi/2Yz_kqxegrs/default.jpg,False,False,False,Troye Sivan revealed to Grimmy that he's worki...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
20000,kA66nAvEEII,18.24.02,Stephen A. Shares Theory On Why Spurs' Kawhi L...,ESPN,17,2018-02-22T15:31:36.000Z,"[espn, ""espn live"", ""first take"", ""espn first""...",707341,4881,290,1569,https://i.ytimg.com/vi/kA66nAvEEII/default.jpg,False,False,False,First Take's Stephen A. Smith shares theory on...
20001,-C_rvt0SwLE,18.24.02,Imagine Dragons - Next To Me (Audio),ImagineDragonsVEVO,10,2018-02-21T17:00:00.000Z,"[Imagine, ""Dragons"", ""Next"", ""To"", ""Me"", ""KIDi...",5010358,279683,3886,17386,https://i.ytimg.com/vi/-C_rvt0SwLE/default.jpg,False,False,False,"Listen to Next To Me, out now: http://smarturl..."
20002,A9YcrloL3oE,18.24.02,Jack,Unbox Therapy,1,2018-02-21T22:29:33.000Z,"[unbox therapy, ""unbox"", ""therapy"", ""jack"", ""u...",3239660,262437,7931,36404,https://i.ytimg.com/vi/A9YcrloL3oE/default.jpg,False,False,False,This wasn't easy for Jack to do... He did it f...
20003,dvJWgErHi7A,18.24.02,Nashville On Cmt | Final Episodes Coming June 7,CMT,24,2018-02-23T03:03:01.000Z,"[nashville, ""nashville on cmt"", ""season 6"", ""f...",40649,90,36,34,https://i.ytimg.com/vi/dvJWgErHi7A/default.jpg,False,False,False,Every journey has a destination. Every song ha...
20004,YBgabneSziQ,18.24.02,Snoop Dogg - One More Day (Audio) Ft. Charlie ...,SnoopDoggVEVO,10,2018-02-23T08:00:02.000Z,"[All The Time Entertainment, ""Gospel/Christian...",81805,3455,196,365,https://i.ytimg.com/vi/YBgabneSziQ/default.jpg,False,False,False,Snoop Dogg- One More Day\n\nAvailable now:http...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,FR5WT12B_88,18.21.03,Spinners,Kaplamino,24,2018-03-17T16:56:40.000Z,"[Chain reaction, ""Rube Goldberg"", ""Fidget spin...",539298,17123,284,1093,https://i.ytimg.com/vi/FR5WT12B_88/default.jpg,False,False,False,I already used two fidget spinners tricks in t...
24996,sfMwXjNo3Rs,18.21.03,What Spring Looks Like Around The World,INSIDER,19,2018-03-17T19:00:04.000Z,"[cherry blossom, ""washington cherry blossom"", ...",382536,7328,217,841,https://i.ytimg.com/vi/sfMwXjNo3Rs/default.jpg,False,False,False,With spring comes a blossom of culture and col...
24997,2XQft9iwSqg,18.21.03,Sing Anything Challenge,Cimorelli,10,2018-03-17T20:47:28.000Z,"[Cimorelli The Cimorellis sisters, ""cimorelli""...",174882,14112,87,2051,https://i.ytimg.com/vi/2XQft9iwSqg/default.jpg,False,False,False,Comment which song was your favorite! Anything...
24998,f1fXCRtSUWU,18.21.03,Top Breeder 🐕,Kensuke Koike,1,2018-03-12T11:15:23.000Z,"[Kensukekoike, ""Todayscuriosity"", ""Collage"", ""...",1308851,22340,908,1921,https://i.ytimg.com/vi/f1fXCRtSUWU/default.jpg,False,False,False,#kensukekoike\n#todayscuriosity\nhttp://www.ke...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
25000,uKkbfxTHJGI,18.21.03,10$ Drum - Faded ( Alan Walker ),Big Marvel,10,2018-03-16T22:45:26.000Z,"[10$ Drum, ""Faded"", ""Alan Walker""]",2481355,84337,9561,6429,https://i.ytimg.com/vi/uKkbfxTHJGI/default.jpg,False,False,False,thank you for watching this video please like ...
25001,hHFuZVGpBC0,18.21.03,Crushing And Slicing Red Hot Steel With Hydrau...,Hydraulic Press Channel,28,2018-03-17T14:55:04.000Z,"[Hydraulic press channel, ""hydraulicpresschann...",551958,10541,398,1089,https://i.ytimg.com/vi/hHFuZVGpBC0/default.jpg,False,False,False,Battle of YouTube Titans Red Hot Steel Vs. Hyd...
25002,zWrMvW7S3tg,18.21.03,The Infinity War Trailer But I Just Name Chara...,jacksfilms,23,2018-03-16T17:16:53.000Z,"[Jacksfilms, ""Jack Douglass"", ""YGS"", ""YGS 100""...",1275395,90583,5240,11788,https://i.ytimg.com/vi/zWrMvW7S3tg/default.jpg,False,False,False,i know him
25003,ffgzh93MdQM,18.21.03,"Keyboard Cat, Bento, A Tribute",Keyboard Cat,15,2018-03-16T15:52:12.000Z,"[Keyboard, ""Cat"", ""Keyboard Cat"", ""Charlie Sch...",373175,12448,100,2321,https://i.ytimg.com/vi/ffgzh93MdQM/default.jpg,False,False,False,https://www.facebook.com/thekeyboardcat\nhttp:...
25004,S5xFft7kg8Q,18.21.03,Adam Scott Hijacks A Stranger'S Tinder | Vanit...,Vanity Fair,24,2018-03-16T16:00:14.000Z,"[adam scott, ""tinder"", ""tinder takeover"", ""hij...",952759,19610,690,1023,https://i.ytimg.com/vi/S5xFft7kg8Q/default.jpg,False,False,False,"On this episode of Tinder Takeover, Adam Scott..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,vRN-32VKEHE,18.21.04,Teens React To Mirror-Polished Japanese Foil B...,FBE,24,2018-04-17T21:00:46.000Z,"[polish, ""polished"", ""smooth"", ""TEENS REACT TO...",1721072,32545,1281,4216,https://i.ytimg.com/vi/vRN-32VKEHE/default.jpg,False,False,False,Mirror-Polished Japanese foil challenge reacte...
29996,hEnWzvzVMaA,18.21.04,Festival Rainbow Makeup & Lookbook,James Charles,24,2018-04-17T21:00:18.000Z,"[james, ""james charles"", ""makeup artist"", ""mua...",3637097,170697,7456,18012,https://i.ytimg.com/vi/hEnWzvzVMaA/default.jpg,False,False,False,HI SISTERS! This past weekend my friends and I...
29997,M84oouivF7k,18.21.04,24 Hours With Camila Cabello: Inside Her First...,Vogue,24,2018-04-18T17:13:16.000Z,"[camilla cabello, ""camila cabello"", ""camila ca...",435338,23194,370,1091,https://i.ytimg.com/vi/M84oouivF7k/default.jpg,False,False,False,We’re all suckers for the way Camila Cabello m...
29998,tWDK9rK-f1Q,18.21.04,Honest Trailers - Baby Driver,Screen Junkies,1,2018-04-17T17:00:04.000Z,"[screenjunkies, ""screen junkies"", ""honest trai...",1759899,43902,1199,4648,https://i.ytimg.com/vi/tWDK9rK-f1Q/default.jpg,False,False,False,You've been obsessed with what Edgar Wright ha...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
30000,-7_ATlZ-zMc,18.21.04,Insta-Justice,Matt Bentkowski,17,2018-04-16T20:37:56.000Z,[[none]],2782387,26055,1328,2237,https://i.ytimg.com/vi/-7_ATlZ-zMc/default.jpg,False,False,False,285 east to 400 traffic being slow as usual an...
30001,Y-R3QdTGeLg,18.21.04,Bodyarmor Sports Drink | James Harden Thanks...,DrinkBODYARMOR,17,2018-04-18T11:44:23.000Z,"[James Harden, ""NBA"", ""NBA Playoffs 2018"", ""BO...",384434,498,57,132,https://i.ytimg.com/vi/Y-R3QdTGeLg/default.jpg,False,False,False,James Harden wouldn't go to the game wearing o...
30002,trcditUaQJg,18.21.04,Marvel Studios' Avengers: Infinity War | 10-Ye...,Marvel Entertainment,24,2018-04-17T21:00:05.000Z,"[marvel, ""comics"", ""comic books"", ""nerdy"", ""ge...",1637917,80961,470,5176,https://i.ytimg.com/vi/trcditUaQJg/default.jpg,False,False,False,It’s all been leading to this. Watch this spec...
30003,S4vxoP-IF2M,18.21.04,How The Squid Lost Its Shell,PBS Eons,28,2018-04-17T21:22:11.000Z,"[dinosaurs, ""dinos"", ""paleo"", ""paleontology"", ...",365063,15651,264,1197,https://i.ytimg.com/vi/S4vxoP-IF2M/default.jpg,False,False,False,"The ancestors of modern, squishy cephalopods l..."
30004,EbbL8NhdTQE,18.21.04,Huge Problem At The New House...,Tanner Fox,1,2018-04-16T19:00:00.000Z,"[tanner fox, ""scootering"", ""funny"", ""music"", ""...",1184846,21900,8796,2026,https://i.ytimg.com/vi/EbbL8NhdTQE/default.jpg,False,False,False,NEW Line just Dropped!! -- http://www.tfoxbra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,mAfkkgw_-68,18.16.05,We Tried To Re-Create This Giant Cinnamon Roll,Tasty,26,2018-05-12T15:00:32.000Z,"[Buzzfeed, ""eating your feed"", ""viral food"", ""...",1330906,30823,653,1629,https://i.ytimg.com/vi/mAfkkgw_-68/default.jpg,False,False,False,“OH MY GOD THEY”RE GIGANTIC!”Check us out on F...
34996,rQEqKZ7CJlk,18.16.05,Bangabandhu Satellite-1 Mission,SpaceX,28,2018-05-11T20:56:59.000Z,[[none]],2951139,49325,5504,6118,https://i.ytimg.com/vi/rQEqKZ7CJlk/default.jpg,False,False,False,SpaceX is targeting launch of Bangabandhu Sate...
34997,OXVm3fhYsEo,18.16.05,Kylie Cosmetics X Kris Jenner Collection | Swa...,Jaclyn Hill,26,2018-05-11T22:25:51.000Z,"[jaclynhill1, ""jaclyn hill"", ""makeup tutorial""...",2455224,101285,5942,12939,https://i.ytimg.com/vi/OXVm3fhYsEo/default.jpg,False,False,False,"*After looking over this footage, I definitely..."
34998,UQkBcHLZOqU,18.16.05,Wearing Fashion Nova Outfits For A Week,Safiya Nygaard,22,2018-05-11T22:36:56.000Z,"[wearing fashion nova outfits for a week, ""fas...",4292255,210812,2318,11519,https://i.ytimg.com/vi/UQkBcHLZOqU/default.jpg,False,False,False,I've been seeing this mysterious hashtag #Nova...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
35000,Am6NHDbj6XA,18.16.05,Donald Glover On This Is America Music Video,Jimmy Kimmel Live,24,2018-05-11T07:30:00.000Z,"[abc, ""network"", ""tv"", ""television"", ""late"", ""...",5437925,93573,11859,3287,https://i.ytimg.com/vi/Am6NHDbj6XA/default.jpg,False,False,False,Donald talks about his huge new video for This...
35001,CPjWgk0UXps,18.16.05,I Got My Apartment Professionally Organized,Michelle Khare,24,2018-05-11T15:00:01.000Z,"[michelle khare, ""buzzfeed michelle"", ""organiz...",1193839,29247,1403,1252,https://i.ytimg.com/vi/CPjWgk0UXps/default.jpg,False,False,False,I worked with professional organizer Ashley Mo...
35002,vjSohj-Iclc,18.16.05,"Getting Some Air, Atlas?",BostonDynamics,28,2018-05-10T19:56:28.000Z,"[Dynamic robots, ""Boston Dynamics"", ""humanoid ...",5936113,78765,2816,12477,https://i.ytimg.com/vi/vjSohj-Iclc/default.jpg,False,False,False,NaN
35003,iILJvqrAQ_w,18.16.05,Charlie Puth - Boy [Official Audio],Charlie Puth,10,2018-05-11T04:00:34.000Z,"[charlie puth, ""boy"", ""charlie"", ""puth"", ""atla...",2492498,84758,1409,4256,https://i.ytimg.com/vi/iILJvqrAQ_w/default.jpg,False,False,False,Voicenotes Available Now: https://Atlantic.lnk...
35004,q1jzwV_s8_Y,18.16.05,Christina Aguilera - Twice (Audio),CAguileraVEVO,10,2018-05-11T07:00:01.000Z,"[Christina Aguilera, ""Pop"", ""RCA Records Label...",2064683,66563,1991,6077,https://i.ytimg.com/vi/q1jzwV_s8_Y/default.jpg,False,False,False,Pre-order Christina Aguilera’s new album ‘Libe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,fAIX12F6958,18.10.06,Bumblebee (2018) - Official Teaser Trailer - P...,Paramount Pictures,1,2018-06-05T07:00:01.000Z,[[none]],18754010,221056,15832,44801,https://i.ytimg.com/vi/fAIX12F6958/default.jpg,False,False,False,Every adventure has a beginning. Watch the off...
39996,_BcYBFC6zfY,18.10.06,Ralph Breaks The Internet: Wreck-It Ralph 2 Of...,Walt Disney Animation Studios,1,2018-06-04T16:00:03.000Z,"[Disney, ""Walt Disney Animation Studios"", ""Dis...",11685298,211632,12663,38538,https://i.ytimg.com/vi/_BcYBFC6zfY/default.jpg,False,False,False,Watch the official trailer for Ralph Breaks th...
39997,v-dfygYIfLs,18.10.06,Old School Trick Shots | Dude Perfect,Dude Perfect,17,2018-06-04T21:55:13.000Z,"[dude perfect, ""dude perfect stereotypes"", ""du...",12893170,430074,9511,37285,https://i.ytimg.com/vi/v-dfygYIfLs/default.jpg,False,False,False,It’s time to look old! Special thanks to the U...
39998,5y0owYFTMno,18.10.06,Ethan Hawke Knows To Seek Knowledge From Masters,The Late Show with Stephen Colbert,24,2018-06-05T07:53:32.000Z,"[The Late Show, ""Late Show"", ""Stephen Colbert""...",344380,3755,214,399,https://i.ytimg.com/vi/5y0owYFTMno/default.jpg,False,False,False,'First Reformed' star Ethan Hawke knows to sta...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40000,2wJESCSvFU8,18.10.06,Live Pd: Can I Text My Mom? (Season 2) | A&E,A&E,24,2018-06-04T12:00:00.000Z,"[a&e, ""a&e (tv network)"", ""aetv"", ""a&e tv"", ""a...",964336,7581,892,2432,https://i.ytimg.com/vi/2wJESCSvFU8/default.jpg,False,False,False,When a K9 hits on a suspect's car he asks the ...
40001,MidgYgFwgTI,18.10.06,Blind Ice Cream Taste Test,Good Mythical Morning,24,2018-06-04T10:00:04.000Z,"[gmm, ""good mythical morning"", ""rhett and link...",1910472,43556,1489,6925,https://i.ytimg.com/vi/MidgYgFwgTI/default.jpg,False,False,False,We're tasting popular vanilla ice creams to de...
40002,J2m7EUwnqGg,18.10.06,Iphone X — Animoji: Taxi Driver — Apple,Apple,28,2018-06-04T04:00:00.000Z,"[Apple, ""iphone"", ""iphone x"", ""animoji"", ""appl...",9032972,78784,31508,0,https://i.ytimg.com/vi/J2m7EUwnqGg/default.jpg,True,False,False,Animoji on iPhone X (feat. HYUKOH) Learn more ...
40003,3uGIEY7tdg8,18.10.06,Suspiria - Teaser Trailer | Amazon Studios,Amazon Studios,1,2018-06-04T13:00:02.000Z,"[Amazon Studios, ""Amazon"", ""amazon prime"", ""am...",2077018,14577,1349,1860,https://i.ytimg.com/vi/3uGIEY7tdg8/default.jpg,False,False,False,A darkness swirls at the center of a world-ren...
40004,v_0WNLAmSaE,18.10.06,Game Theory: Fnaf Stumped Me! (Fnaf 6 Ultimate...,The Game Theorists,20,2018-06-03T18:25:05.000Z,"[fnaf, ""five nights"", ""New Fnaf"", ""custom nigh...",4656429,200994,8055,90979,https://i.ytimg.com/vi/v_0WNLAmSaE/default.jpg,False,False,False,Bendy FOOLED Us! ► https://bit.ly/2JaTzWiFNAF...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught The Laser,AaronsAnimals,15,2018-05-18T13:00:04.000Z,"[aarons animals, ""aarons"", ""animals"", ""cat"", ""...",1685609,38160,1385,2657,https://i.ytimg.com/vi/BZt0qjTWNhw/default.jpg,False,False,False,The Cat Who Caught the Laser - Aaron's Animals
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18T01:00:06.000Z,[[none]],1064798,60008,382,3936,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,NaN
40946,D6Oy4LfoqsU,18.14.06,I Gave Safiya Nygaard A Perfect Hair Makeover ...,Brad Mondo,24,2018-05-18T17:34:22.000Z,[I gave safiya nygaard a perfect hair makeover...,1066451,48068,1032,3992,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17T17:00:04.000Z,"[Black Panther, ""HISHE"", ""Marvel"", ""Infinity W...",5660813,192957,2846,13088,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...


### Technique No. 3: Make numeric types as small as possible

**NOTE**: It is better to use this technique along with the above methods...

In [19]:
df = pd.read_csv('./datasets/youtube_trends_us.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

In [20]:
df.select_dtypes(include=['number']).agg(['min', 'max'])

,category_id,views,likes,dislikes,comment_count
min,1,549,0,0,0
max,43,225211923,5613827,1674420,1361580


In [21]:
df = df.astype({'category_id': 'int8'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int8  
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

In [23]:
dtypes = {
    'category_id': 'int8',
    'views': 'int32',
    'likes': 'int32',
    'dislikes': 'int32',
    'comment_count': 'int32',
}

In [25]:
df = pd.read_csv('./datasets/youtube_trends_us.csv', dtype=dtypes)
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int8  
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int32 
 8   likes                   40949 non-null  int32 
 9   dislikes                40949 non-null  int32 
 10  comment_count           40949 non-null  int32 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de